In [ ]:
!python --version

In [ ]:
import tensorflow as tf
#import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# import du dataset
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv('../../dataset/order_classification/edible_mushrooms.csv')

# Generating data

In [ ]:
path='../../dataset/class_order'
train_path='../../dataset/order_classification/train'
test_path='../../dataset/order_classification/test'
validation_path='../../dataset/order_classification/validation' 

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
train_generator = train_data_generator.flow_from_directory(
    directory = train_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
test_generator = test_data_generator.flow_from_directory(
    directory = test_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
validation_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
validation_generator = validation_data_generator.flow_from_directory(
    directory = validation_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

# DensetNet transfer learning

In [ ]:

import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
n_class= 8

base_model = DenseNet121(weights = 'imagenet', include_top = False, classifier_activation = None,  input_shape=(224,224,3))

# Get the output of the base model
base_output = base_model.output

for layer in base_model.layers:
    layer.trainable = False

# Add your custom layers
x = GlobalAveragePooling2D()(base_output)  # Flatten layer
x = Dense(1024, activation='relu')(x)  # Dense layer
x = Dropout(rate=0.2)(x)
x = Dense(n_class, activation='softmax')(x)  # Dense layer

# Get the output of the base model
base_output = base_model.output

# Create a new model that includes the base model and your custom layers
model_densenet = Model(inputs=base_model.input, outputs=x)

In [ ]:
model_densenet.compile(optimizer='adam', 
               loss='sparse_categorical_crossentropy', 
               metrics=['acc'])
model_densenet.summary()

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"  # specify which GPU(s) to be used

# Create an early stopping callback
early_stopping = EarlyStopping(monitor='val_acc', patience=5, min_delta = 0.001)

model_densenet_history = model_densenet.fit(
    train_generator,
    epochs = 20,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

In [ ]:
# Save the model
model_densenet.save('./model_densenet_checkpoint')

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_densenet_history.history['acc'])
plt.plot(model_densenet_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()